In [2]:
import numpy as np
import pandas as pd
import scipy.stats as st
import time
import matplotlib.pyplot as plt
%matplotlib inline
title = 'woplus'
path = '../../sources/data'

trainset_name = 'fe3_2-2-1'

In [4]:
df_train_f = pd.read_hdf('../../sources/deal/{}.h5'.format(trainset_name), key = trainset_name)
# 计算训练集正负样本比例
st = df_train_f.Flag.value_counts()
st.div(st.sum())

0    0.818865
1    0.181135
Name: Flag, dtype: float64

In [5]:
df_train_f.head()
# df_train_f.iloc[:, 10].plot(kind='kde')

,net,age_17岁以下,age_18-22,age_23-25,age_26-29,age_30-39,age_40-49,age_50-59,age_60以上,sex_不详,...,"salestimediff_cat_(1440, 1800]","salestimediff_cat_(1800, 2160]","salestimediff_cat_(2160, 2520]","salestimediff_cat_(2520, 2880]","salestimediff_cat_(2880, 3240]","salestimediff_cat_(3240, 3600]","salestimediff_cat_(3600, 3960]","salestimediff_cat_(3960, 4320]","salestimediff_cat_(4320, 4680]",Flag
0,-1.285664,-0.210748,-0.169149,-0.198993,-0.28939,1.241278,-0.480494,-0.385466,-0.340289,-0.139559,...,-0.236911,-0.147399,-0.089668,-0.042028,-0.064122,-0.071056,-0.048687,-0.004079,-0.001665,0
1,0.777808,-0.210748,-0.169149,-0.198993,-0.28939,1.241278,-0.480494,-0.385466,-0.340289,-0.139559,...,-0.236911,-0.147399,-0.089668,-0.042028,-0.064122,-0.071056,-0.048687,-0.004079,-0.001665,0
2,0.777808,-0.210748,-0.169149,-0.198993,-0.28939,1.241278,-0.480494,-0.385466,-0.340289,-0.139559,...,-0.236911,-0.147399,-0.089668,-0.042028,-0.064122,-0.071056,-0.048687,-0.004079,-0.001665,0
3,0.777808,-0.210748,-0.169149,-0.198993,-0.28939,1.241278,-0.480494,-0.385466,-0.340289,-0.139559,...,-0.236911,-0.147399,-0.089668,-0.042028,-0.064122,-0.071056,-0.048687,-0.004079,-0.001665,0
4,0.777808,-0.210748,-0.169149,-0.198993,-0.28939,1.241278,-0.480494,-0.385466,-0.340289,-0.139559,...,4.220987,-0.147399,-0.089668,-0.042028,-0.064122,-0.071056,-0.048687,-0.004079,-0.001665,0


In [15]:
m, n = np.shape(df_train_f)
X = df_train_f.iloc[:, :(n-1)]
y = df_train_f.iloc[:, -1]
irx = list(X.index)

In [16]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve, roc_curve, auc, roc_auc_score, accuracy_score

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.3, random_state=0)
print df_train_f.shape
print np.shape(X_train)
print np.shape(X_test)
print np.shape(X_test)[0] * 1.0 / (np.shape(X_train)[0]+np.shape(X_test)[0])

(721396, 119)
(504977, 118)
(216419, 118)
0.30000027724


## 基模型组

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb

models = [
    ['lr', LogisticRegression(random_state=0)], 
    ['rf', RandomForestClassifier(oob_score=True, random_state=0)],
    ['gbdt', GradientBoostingClassifier(random_state=0)],
    ['xgbc', xgb.XGBClassifier(nthread=-1, seed=0)]
]

## Blending

In [18]:
# blending
# 切分训练集为两部分d1 d2
X_d1, X_d2, y_d1, y_d2 = train_test_split(X_train, y_train, test_size=0.5, random_state=0)

In [20]:
# 用d1训练模型
blending_d2      = np.zeros((X_d2.shape[0], len(models)))
blending_test = np.zeros((X_test.shape[0], len(models)))

In [22]:
# for every base model
for j, (name, model) in enumerate(models):
    timeStart = time.time()
    model.fit(X_d1, y_d1)
    print 'No.{} - {} - Training Time: {:.2f} seconds'.format(j+1, name, time.time() - timeStart)
    blending_d2[:, j]  = model.predict_proba(X_d2)[:, 1]
    blending_test[:, j] = model.predict_proba(X_test)[:, 1]
    print 'base model %s: testSet auc Score = %.6f' % (name,  roc_auc_score(y_test, blending_test[:, j]))

No.1 - lr - Training Time: 43.33 seconds
base model lr: testSet auc Score = 0.741998


/Users/fanghan/anaconda/lib/python2.7/site-packages/sklearn/ensemble/forest.py:451: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/fanghan/anaconda/lib/python2.7/site-packages/sklearn/ensemble/forest.py:456: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


No.2 - rf - Training Time: 8.26 seconds
base model rf: testSet auc Score = 0.761818
No.3 - gbdt - Training Time: 213.04 seconds
base model gbdt: testSet auc Score = 0.795276
No.4 - xgbc - Training Time: 29.05 seconds
base model xgbc: testSet auc Score = 0.795163


In [24]:
# blending 融合
clf = LogisticRegression()
clf.fit(blending_d2, y_d2)
clf_predict_prob = clf.predict_proba(blending_test)[:, 1]

# print("Linear stretch of predictions to [0,1]")
clf_predict_prob_stretch = (clf_predict_prob - clf_predict_prob.min()) / (clf_predict_prob.max() - clf_predict_prob.min())

print 'blending result( no  stretch): testSet auc Score = %.6f' % (roc_auc_score(y_test, clf_predict_prob))
print 'blending result(with stretch): testSet auc Score = %.6f' % (roc_auc_score(y_test, clf_predict_prob_stretch))

blending result( no  stretch): testSet auc Score = 0.795647
blending result(with stretch): testSet auc Score = 0.795647


## Stacking

In [25]:
# 不再切分训练集为两部分d1 d2，而是直接做nfolds
# X_d1, X_d2, y_d1, y_d2 = train_test_split(X, y, test_size=0.5, random_state=0)

skf = StratifiedKFold(n_splits=5, shuffle=False, random_state=0)
# 训练模型
stacking_train   = np.zeros((X_train.shape[0], len(models)))
stacking_predict = np.zeros((X_test.shape[0], len(models)))

In [31]:
# for every base model
for j, (name, model) in enumerate(models):
    stacking_predict_j = np.zeros((X_test.shape[0], 5))
    for i, (train, test) in enumerate(skf.split(X_train, y_train)):
        Xtrain, Xtest, ytrain, ytest = X_train[train], X_train[test], y_train[train], y_train[test]
        timeStart = time.time()
        model.fit(Xtrain, ytrain)
        print 'No.{} - {} - Training Time: {:.2f} seconds'.format(j+1, name, time.time() - timeStart)
        stacking_train[test, j]  = model.predict_proba(Xtest)[:, 1]
        stacking_predict_j[:, i] = model.predict_proba(X_test)[:, 1]
    stacking_predict[:, j] = stacking_predict_j.mean(axis=1)
    print 'base model %s: testSet auc Score = %.6f' % (name,  roc_auc_score(y_test, stacking_predict[:, j]))

No.1 - lr - Training Time: 75.11 seconds
No.1 - lr - Training Time: 72.41 seconds
No.1 - lr - Training Time: 58.52 seconds
No.1 - lr - Training Time: 50.21 seconds
No.1 - lr - Training Time: 55.03 seconds
base model lr: testSet auc Score = 0.743891
No.2 - rf - Training Time: 15.11 seconds
No.2 - rf - Training Time: 14.92 seconds
No.2 - rf - Training Time: 15.20 seconds
No.2 - rf - Training Time: 15.35 seconds
No.2 - rf - Training Time: 15.11 seconds
base model rf: testSet auc Score = 0.782521
No.3 - gbdt - Training Time: 361.61 seconds
No.3 - gbdt - Training Time: 369.18 seconds
No.3 - gbdt - Training Time: 358.56 seconds
No.3 - gbdt - Training Time: 363.21 seconds
No.3 - gbdt - Training Time: 376.61 seconds
base model gbdt: testSet auc Score = 0.795546
No.4 - xgbc - Training Time: 48.36 seconds
No.4 - xgbc - Training Time: 48.63 seconds
No.4 - xgbc - Training Time: 47.94 seconds
No.4 - xgbc - Training Time: 48.03 seconds
No.4 - xgbc - Training Time: 47.56 seconds
base model xgbc: test

In [30]:
# stacking 融合
clf = LogisticRegression()
clf.fit(stacking_train, y_train)
clf_predict_prob = clf.predict_proba(stacking_predict)[:, 1]

# print("Linear stretch of predictions to [0,1]")
clf_predict_prob_stretch = (clf_predict_prob - clf_predict_prob.min()) / (clf_predict_prob.max() - clf_predict_prob.min())

print 'stacking result( no  stretch): testSet auc Score = %.6f' % (roc_auc_score(y_test, clf_predict_prob))
print 'stacking result(with stretch): testSet auc Score = %.6f' % (roc_auc_score(y_test, clf_predict_prob_stretch))

stacking result( no  stretch): testSet auc Score = 0.798388
stacking result(with stretch): testSet auc Score = 0.798388
